# Install necessary packages

In [18]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


# Part 3: Practical Data Preparation

**Objective:** Handle categorical features using One-Hot Encoding and address class imbalance using SMOTE.

## 1. Setup

Import necessary libraries.


In [19]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE


## 2. Data Loading

Load the dataset.

In [20]:
def load_data(file_path):
    """
    Load the synthetic health data from a CSV file.
    
    Args:
        file_path: Path to the CSV file
        
    Returns:
        DataFrame containing the data
    """
    # YOUR CODE HERE
    # Load the CSV file using pandas
    df = pd.read_csv(file_path)
    
    return df  # Replace with actual implementation

## 3. Categorical Feature Encoding

Implement `encode_categorical_features` using `OneHotEncoder`.

In [21]:
def encode_categorical_features(df, column_to_encode='smoker_status'):
    """
    Encode a categorical column using OneHotEncoder.
    
    Args:
        df: Input DataFrame
        column_to_encode: Name of the categorical column to encode
        
    Returns:
        DataFrame with the categorical column replaced by one-hot encoded columns
    """
    # YOUR CODE HERE
    # 1. Extract the categorical column
    categorical_col = df[[column_to_encode]]
    # 2. Apply OneHotEncoder
    encoder = OneHotEncoder(drop='first')  
    encoded = encoder.fit_transform(categorical_col).toarray()
    # 3. Create new column names
    encoded_cols = encoder.get_feature_names_out([column_to_encode])
    # 4. Replace the original categorical column with the encoded columns
    df_encoded = df.drop(columns=[column_to_encode]).copy()
    df_encoded[encoded_cols] = encoded
    # Placeholder return - replace with your implementation
    return df_encoded

## 4. Data Preparation

Implement `prepare_data_part3` to handle the train/test split correctly.


In [22]:
def prepare_data_part3(df, test_size=0.2, random_state=42):
    """
    Prepare data with categorical encoding.
    
    Args:
        df: Input DataFrame
        test_size: Proportion of data for testing
        random_state: Random seed for reproducibility
        
    Returns:
        X_train, X_test, y_train, y_test
    """
    # YOUR CODE HERE
    # 1. Encode categorical features using the encode_categorical_features function
    df_encoded = encode_categorical_features(df, column_to_encode='smoker_status')
    df_encoded = df_encoded.drop(columns=['timestamp'])
    # 2. Select relevant features (including the one-hot encoded ones) and the target
    X = df_encoded.drop(columns='disease_outcome')
    y = df_encoded['disease_outcome']
    # 3. Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    # 4. Handle missing values
    numeric_cols = X.select_dtypes(include=['number']).columns
    imputer = SimpleImputer(strategy='mean')

    # Apply imputation only to numeric columns
    X_train[numeric_cols] = imputer.fit_transform(X_train[numeric_cols])
    X_test[numeric_cols] = imputer.transform(X_test[numeric_cols])
    
    # Placeholder return - replace with your implementation
    return X_train, X_test, y_train, y_test

## 5. Handling Imbalanced Data

Implement `apply_smote` to oversample the minority class.

In [23]:
def apply_smote(X_train, y_train, random_state=42):
    """
    Apply SMOTE to oversample the minority class.
    
    Args:
        X_train: Training features
        y_train: Training target
        random_state: Random seed for reproducibility
        
    Returns:
        Resampled X_train and y_train with balanced classes
    """
    # YOUR CODE HERE
    # Apply SMOTE to balance the classes
    smote = SMOTE(random_state=random_state)
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
    # Placeholder return - replace with your implementation
    
    return X_resampled, y_resampled

## 6. Model Training and Evaluation

Train a model on the SMOTE-resampled data and evaluate it.

In [24]:
def train_logistic_regression(X_train, y_train):
    """
    Train a logistic regression model.
    
    Args:
        X_train: Training features
        y_train: Training target
        
    Returns:
        Trained logistic regression model
    """
    # YOUR CODE HERE
    # Initialize and train a LogisticRegression model
    model = LogisticRegression(max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    return model  # Replace with actual implementation

def calculate_evaluation_metrics(model, X_test, y_test):
    """
    Calculate classification evaluation metrics.
    
    Args:
        model: Trained model
        X_test: Test features
        y_test: Test target
        
    Returns:
        Dictionary containing accuracy, precision, recall, f1, auc, and confusion_matrix
    """
    # YOUR CODE HERE
    # 1. Generate predictions
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    # 2. Calculate metrics: accuracy, precision, recall, f1, auc
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)
    
    # 3. Create confusion matrix
    cm = confusion_matrix(y_test, y_pred).tolist
    # 4. Return metrics in a dictionary
    
    # Placeholder return - replace with your implementation
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc': auc,
        'confusion_matrix': cm
    }

## 7. Save Results

Save the evaluation metrics to a text file.

In [25]:
# YOUR CODE HERE
def save_metrics_to_file(metrics, filename='results/results_part3.txt'):
# 1. Create 'results' directory if it doesn't exist
    os.makedirs(os.path.dirname(filename), exist_ok=True)
# 2. Format metrics as strings
    formatted_lines = [
        f"{key}: {value:.4f}" if np.isscalar(value)
        else f"{key}: {np.array2string(np.array(value), precision=4)}"
        for key, value in metrics.items()
    ]


# 3. Write metrics to 'results/results_part3.txt'
    with open(filename, 'w') as f:
        for line in formatted_lines:
            f.write(line + '\n')

In [26]:
def load_metrics_from_text(filepath):
    metrics = {}
    with open(filepath, 'r') as f:
        lines = f.read().strip().splitlines()

    i = 0
    while i < len(lines):
        line = lines[i]
        if ':' not in line:
            i += 1
            continue
        key, val = line.split(':', 1)
        key = key.strip()
        val = val.strip()
        if key == 'confusion_matrix':
            matrix = []
            i += 1
            while i < len(lines) and lines[i].startswith('['):
                row = eval(lines[i])
                matrix.append(row)
                i += 1
            metrics[key] = matrix
        else:
            metrics[key] = float(val)
            i += 1
    return metrics


## 9. Compare Results

Implement a function to compare model performance between balanced and imbalanced data.


In [27]:
def compare_models(part1_metrics, part3_metrics):
    """
    Calculate percentage improvement between models trained on imbalanced vs. balanced data.
    
    Args:
        part1_metrics: Dictionary containing evaluation metrics from Part 1 (imbalanced)
        part3_metrics: Dictionary containing evaluation metrics from Part 3 (balanced)
        
    Returns:
        Dictionary with metric names as keys and improvement percentages as values
    """
    # YOUR CODE HERE
    improvement = {}
    # 1. Calculate percentage improvement for each metric
    for metric in ['accuracy', 'precision', 'recall', 'f1', 'auc']:
        val1 = part1_metrics.get(metric)
        val3 = part3_metrics.get(metric)

    # 2. Handle metrics where higher is better (most metrics) and where lower is better
        if isinstance(val1, (int, float)) and isinstance(val3, (int, float)) and val1 != 0:
            # 3. Calculate percentage improvement
            improvement[metric] = ((val3 - val1) / abs(val1)) * 100
        else:
            improvement[metric] = 0.0  # Fallback if invalid or zero baseline
    # 3. Return a dictionary with metric names and improvement percentages
    
    # Placeholder return - replace with your implementation
    return improvement

## 8. Main Execution

Run the complete workflow.

In [28]:
# Main execution
if __name__ == "__main__":
    # 1. Load data
    data_file = 'data/synthetic_health_data.csv'
    df = load_data(data_file)
    
    # 2. Prepare data with categorical encoding
    X_train, X_test, y_train, y_test = prepare_data_part3(df)
    
    # 3. Apply SMOTE to balance the training data
    X_train_resampled, y_train_resampled = apply_smote(X_train, y_train)
    
    # 4. Train model on resampled data
    model = train_logistic_regression(X_train_resampled, y_train_resampled)
    
    # 5. Evaluate on original test set
    metrics = calculate_evaluation_metrics(model, X_test, y_test)
    
    # 6. Print metrics
    for metric, value in metrics.items():
        if metric != 'confusion_matrix':
            print(f"{metric}: {value:.4f}")
    
    # 7. Save results
    save_metrics_to_file(metrics)
    
    # 8. Load Part 1 results for comparison
#    try:
#        with open('results/results_part1.txt', 'r') as f:
#            content = f.read()
#            part1_metrics = json.loads(content)
    try:
        part1_metrics = load_metrics_from_text('results/results_part1.txt')
    
        # 9. Compare models
        comparison = compare_models(part1_metrics, metrics)
        print("\nModel Comparison (improvement percentages):")
        for metric, improvement in comparison.items():
            print(f"{metric}: {improvement:.2f}%")
    except FileNotFoundError:
        print("Part 1 results not found. Run part1_introduction.ipynb first.")

accuracy: 0.8547
precision: 0.3885
recall: 0.8531
f1_score: 0.5339
auc: 0.9267

Model Comparison (improvement percentages):
accuracy: -6.77%
precision: -41.26%
recall: 183.72%
f1: 0.00%
auc: 2.01%
